### Using PySpark and a RandomForestClassifier's Feature Importances to Understand Spoilage in Dog Food

By: Matt Purvis

The scenario: Purina Dog Food wants to investigate four preservatives (A,B,C,D) that may be causing their dog food to spoil. This project will train a random forest classifier and use its' feature importances to figure out which preservative(s) are most responsible for the spoiled batches of food.

In [0]:
# Import SparkSession and create the session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree').getOrCreate()

In [0]:
# Import the data using SparkSQL
data = spark.sql('select * from dog_food_csv')

In [0]:
# Look at the schemas to get column names and types
data.printSchema()

root
-- A: integer (nullable = true)
-- B: integer (nullable = true)
-- C: double (nullable = true)
-- D: integer (nullable = true)
-- Spoiled: double (nullable = true)

In [0]:
# Preview the data
data.show()

+---+---+----+---+-------+
 A| B| C| D|Spoiled|
+---+---+----+---+-------+
 4| 2|12.0| 3| 1.0|
 5| 6|12.0| 7| 1.0|
 6| 2|13.0| 6| 1.0|
 4| 2|12.0| 1| 1.0|
 4| 2|12.0| 3| 1.0|
 10| 3|13.0| 9| 1.0|
 8| 5|14.0| 5| 1.0|
 5| 8|12.0| 8| 1.0|
 6| 5|12.0| 9| 1.0|
 3| 3|12.0| 1| 1.0|
 9| 8|11.0| 3| 1.0|
 1| 10|12.0| 3| 1.0|
 1| 5|13.0| 10| 1.0|
 2| 10|12.0| 6| 1.0|
 1| 10|11.0| 4| 1.0|
 5| 3|12.0| 2| 1.0|
 4| 9|11.0| 8| 1.0|
 5| 1|11.0| 1| 1.0|
 4| 9|12.0| 10| 1.0|
 5| 8|10.0| 9| 1.0|
+---+---+----+---+-------+
only showing top 20 rows

In [0]:
# Import the vectorassembler to transform the data to the correct format
from pyspark.ml.feature import VectorAssembler

In [0]:
# Look at column names to use for the vector assembler
data.columns

Out[6]: ['A', 'B', 'C', 'D', 'Spoiled']

In [0]:
# Create the vector assembler object, using the 4 preservatives as the features
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'],outputCol='features')

In [0]:
# Transform the data using the assembler object
output = assembler.transform(data)

In [0]:
# Show the resulting transformed dataframe
output.show()

+---+---+----+---+-------+-------------------+
 A| B| C| D|Spoiled| features|
+---+---+----+---+-------+-------------------+
 4| 2|12.0| 3| 1.0| [4.0,2.0,12.0,3.0]|
 5| 6|12.0| 7| 1.0| [5.0,6.0,12.0,7.0]|
 6| 2|13.0| 6| 1.0| [6.0,2.0,13.0,6.0]|
 4| 2|12.0| 1| 1.0| [4.0,2.0,12.0,1.0]|
 4| 2|12.0| 3| 1.0| [4.0,2.0,12.0,3.0]|
 10| 3|13.0| 9| 1.0|[10.0,3.0,13.0,9.0]|
 8| 5|14.0| 5| 1.0| [8.0,5.0,14.0,5.0]|
 5| 8|12.0| 8| 1.0| [5.0,8.0,12.0,8.0]|
 6| 5|12.0| 9| 1.0| [6.0,5.0,12.0,9.0]|
 3| 3|12.0| 1| 1.0| [3.0,3.0,12.0,1.0]|
 9| 8|11.0| 3| 1.0| [9.0,8.0,11.0,3.0]|
 1| 10|12.0| 3| 1.0|[1.0,10.0,12.0,3.0]|
 1| 5|13.0| 10| 1.0|[1.0,5.0,13.0,10.0]|
 2| 10|12.0| 6| 1.0|[2.0,10.0,12.0,6.0]|
 1| 10|11.0| 4| 1.0|[1.0,10.0,11.0,4.0]|
 5| 3|12.0| 2| 1.0| [5.0,3.0,12.0,2.0]|
 4| 9|11.0| 8| 1.0| [4.0,9.0,11.0,8.0]|
 5| 1|11.0| 1| 1.0| [5.0,1.0,11.0,1.0]|
 4| 9|12.0| 10| 1.0|[4.0,9.0,12.0,10.0]|
 5| 8|10.0| 9| 1.0| [5.0,8.0,10.0,9.0]|
+---+---+----+---+-------+-------------------+
only showing top 20 rows

In [0]:
# Get only the features and label columns
final_data = output.select('features', 'Spoiled')

In [0]:
# Preview the data
final_data.show()

+-------------------+-------+
 features|Spoiled|
+-------------------+-------+
 [4.0,2.0,12.0,3.0]| 1.0|
 [5.0,6.0,12.0,7.0]| 1.0|
 [6.0,2.0,13.0,6.0]| 1.0|
 [4.0,2.0,12.0,1.0]| 1.0|
 [4.0,2.0,12.0,3.0]| 1.0|
[10.0,3.0,13.0,9.0]| 1.0|
 [8.0,5.0,14.0,5.0]| 1.0|
 [5.0,8.0,12.0,8.0]| 1.0|
 [6.0,5.0,12.0,9.0]| 1.0|
 [3.0,3.0,12.0,1.0]| 1.0|
 [9.0,8.0,11.0,3.0]| 1.0|
[1.0,10.0,12.0,3.0]| 1.0|
[1.0,5.0,13.0,10.0]| 1.0|
[2.0,10.0,12.0,6.0]| 1.0|
[1.0,10.0,11.0,4.0]| 1.0|
 [5.0,3.0,12.0,2.0]| 1.0|
 [4.0,9.0,11.0,8.0]| 1.0|
 [5.0,1.0,11.0,1.0]| 1.0|
[4.0,9.0,12.0,10.0]| 1.0|
 [5.0,8.0,10.0,9.0]| 1.0|
+-------------------+-------+
only showing top 20 rows

In [0]:
# Import the randomforestclassifier and create the object that will be used to train on the data
from pyspark.ml.classification import RandomForestClassifier
rfc = RandomForestClassifier(numTrees = 150, labelCol = 'Spoiled', featuresCol = 'features')

#### Fit the model on all the data

We do not need train/test split because we do not need the predictions in this scenario. We will not deploy the model. We are only interested in seeing the preservative(s) most responsible for the spoiled food

In [0]:
# Fit the model
rfc_model = rfc.fit(final_data)

In [0]:
# Grab the coefficients of all the preservatives
rfc_model.featureImportances

Out[21]: SparseVector(4, {0: 0.0205, 1: 0.019, 2: 0.9381, 3: 0.0224})

Preservative C is the most important predictor for spoiled dog food. It is recommended to be reduced or taken out of the formula completely to prevent dog food from future spoilage